In [22]:
import numpy as np
import pandas as pd
import csv
import os

## Greedy Algorithm

In [23]:
n = 10 # number of campaigns
m = 100000 # number of customers

s = np.random.choice([0, 1, 2], n) # single slot
d = np.random.choice([5, 8, 15, 17, 23, 30], n) # processing time
c = np.random.randint(low = 10, high = 100, size = n) # cost
b = np.random.randint(low = 70, high = 300, size = n) # benefit
C = np.random.randint(low = 0.4*m, high = 0.95*m, size = n)*c # maximum cost

zippedList =  list(zip(s, d, c, b, C))
campaigns = pd.DataFrame(zippedList, columns = ['starting time', 'processing time' , 'cost', 'benefit', 'maximum cost'])
campaigns.index.names = ['name']
# print("Dataframe : ",campaigns, sep='\n')

# matrix with take-rates
T = np.random.random_sample((m, n))

In [24]:
def greedy(n, campaigns, take_rates):
    campaigns['ratio'] = campaigns['benefit'] / campaigns['cost']
    sorted_campaigns = campaigns.sort_values(by=['ratio'], ascending=False)
    sorted_campaigns['total cost'] = 0
    customers = pd.DataFrame(take_rates)
    customers['campaigns'] = [[] for _ in range(len(customers))] # list to store assigned campaigns
    customers['blocking times'] = [set() for _ in range(len(customers))] # list to store blocking times
    customers.index.names = ['costumer']
    profit  = 0
    
    for j in range(len(sorted_campaigns)):
        current_campaign = sorted_campaigns.index[j]
        sorted_customers = customers.sort_values(by=[current_campaign], ascending=False)
        start_time = sorted_campaigns.loc[current_campaign, 'starting time']
        end_time = sorted_campaigns.loc[current_campaign, 'starting time'] + sorted_campaigns.loc[current_campaign, 'processing time']
        for i in range(len(sorted_customers)):
            current_costumer = sorted_customers.index[i]
            if sorted_campaigns.loc[current_campaign, 'total cost'] + sorted_campaigns.loc[current_campaign, 'cost'] <= sorted_campaigns.loc[current_campaign, 'maximum cost']:
                if start_time not in customers.loc[current_costumer, 'blocking times']:
                    sorted_campaigns.loc[current_campaign, 'total cost'] = sorted_campaigns.loc[current_campaign, 'cost'] + sorted_campaigns.loc[current_campaign, 'total cost']
                    customers.loc[current_costumer, 'campaigns'].append(current_campaign)
                    customers.loc[current_costumer, 'blocking times'].update(range(start_time,end_time))
                    profit += (sorted_campaigns.loc[current_campaign, 'benefit'] * customers.loc[current_costumer, current_campaign]) - sorted_campaigns.loc[current_campaign, 'cost']
            else:
                break
                
    return customers['campaigns'], profit

In [26]:
schedule, profit = greedy(m, campaigns ,T)

cwd = os.getcwd()
path = '{}/output'.format(cwd)
if not os.path.exists(path):
    os.makedirs(path)
schedule.to_csv('{}/greedy.csv'.format(path))